# 1고지

In [2]:
import numpy as np

In [20]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    # 재귀 방식의 역전파
    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input
    #         x.grad = f.backward(self.grad)
    #         x.backward()

    # 반복문을 이용한 역전파
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [21]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)

        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [22]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [23]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [24]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gy = np.exp(x) * gy
        return gy

In [25]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [26]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

In [27]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [28]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

TypeError: <class 'numpy.float64'> is not supported.

In [29]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [30]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [31]:
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward

In [32]:
print(x.grad)

<bound method Square.backward of <__main__.Square object at 0x00000226279A3670>>


In [33]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [34]:
def square(x):
    f = Square()
    return f(x)
    # return Square()(x)

def exp(x):
    f = Exp()
    return f(x)
    # return Exp()(x)

In [35]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [36]:
x = Variable(np.array(0.5))
x = Variable(None)

x = Variable(1.0)

TypeError: <class 'float'> is not supported.

In [37]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

# 2고지

In [38]:
import weakref

class Function2:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable2(as_array(y)) for y in ys]
        
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [39]:
class Add(Function2):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [45]:
def as_variable(obj):
    if isinstance(obj, Variable2):
        return obj
    return Variable2(obj)

In [46]:
# 가변인자가 아닌 리스트 형태로 값을 받았을 때 (클래스 내부 변경한 상태라서 ★오류 생성)
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

TypeError: <class 'list'> is not supported.

In [47]:
# Variable2 Backup
class Variable2:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    # 재귀 방식의 역전파
    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input
    #         x.grad = f.backward(self.grad)
    #         x.backward()

    # 반복문을 이용한 역전파
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

In [50]:
# 가변인자를 parameter로 받아서 계산
x0 = Variable2(np.array(2))
x1 = Variable2(np.array(3))

f = Add()

y = f(x0, x1)
print(y.data)

5


In [51]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [52]:
x0 = Variable2(np.array(2))
x1 = Variable2(np.array(3))

y = add(x0, x1)
print(y.data)

5


In [53]:
# Variable2 work-space
class Variable2:
    def __init__(self, data, name = None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p}) from __repr__'

    def __mul__(self, other):
        return mul(self, other)

    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
        
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

In [54]:
class Square(Function2):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [55]:
def square(x):
    f = Square()
    return f(x)
    # return Square()(x)

In [56]:
x = Variable2(np.array(2.0))
y = Variable2(np.array(3.0))

z = add(square(x), square(y))
z.backward()

In [57]:
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [58]:
x = Variable2(np.array(3.0))
y = add(x, x)
y.backward()
print(y.grad)

None


In [59]:
x = Variable2(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


In [60]:
x = Variable2(np.array(2.0))         
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


In [61]:
x0 = Variable2(np.array(1.0))
x1 = Variable2(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [62]:
class Config:
    enable_backprop = True

In [63]:
Config.enable_backprop = True
x = Variable2(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()
print(x0.grad, x1.grad)
# %memit # peak memory: 138.91 MiB, increment: 0.00 MiB

2.0 1.0


In [64]:
# Config.enable_backprop = False
# x = Variable2(np.ones((100, 100, 100)))
# y = square(square(square(x)))
# # y.backward() # False일 경우 backward 사용 불가

In [65]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    print(f"*** Mode = {value}")
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)
        print("*** Done")

In [66]:
with using_config('enable_backprop', False):
    x = Variable2(np.array(2.0))
    y = square(x)
    print("y:", y.data)
    # %memit # peak memory: 100.75 MiB, increment: 0.00 MiB

*** Mode = False
y: 4.0
*** Done


In [67]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable2(np.array(2.0))
    y = square(x)
    # %memit # peak memory: 100.71 MiB, increment: 0.00 MiB

*** Mode = False
*** Done


#### contextlib 예제

In [68]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield
    finally:
        print('done')

with config_test():
    print('process...')

start
process...
done


In [69]:
# !pip install memory_profiler

In [70]:
# %load_ext memory_profiler

In [71]:
# %memit

#### Mul Class

In [72]:
class Mul(Function2):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1*gy, x0*gy

In [73]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [74]:
a = Variable2(np.array(3.))
b = Variable2(np.array(2.))
c = Variable2(np.array(1.))

y = add(mul(a, b), c)
y.backward()

In [75]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0) from __repr__
2.0
3.0


In [76]:
a = Variable2(np.array(3.))
b = Variable2(np.array(2.))

y = a * b
print(y)

variable(6.0) from __repr__


In [77]:
Variable2.__mul__ = mul
Variable2.__add__ = add
Variable2.__rmul__ = mul
Variable2.__radd__ = add

In [78]:
a = Variable2(np.array(3.))
b = Variable2(np.array(2.))
c = Variable2(np.array(1.))

y = a * b + c
y.backward()

In [79]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0) from __repr__
2.0
3.0


In [80]:
def as_variable(obj):
    if isinstance(obj, Variable2):
        return obj
    return Variable2(obj)

In [81]:
x = Variable2(np.array(2.))
y = x + np.array(3.0)
print(y)

variable(5.0) from __repr__


In [82]:
x = Variable2(np.array(2.))
y = x + 3.0
print(y)

variable(5.0) from __repr__


In [83]:
y = 2.0 * x + 1.0
print(y)

variable(5.0) from __repr__


#### 기타 함수 및 메서드 추가

In [84]:
class Neg(Function2):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy

In [85]:
def neg(x):
    return Neg()(x)

In [86]:
Variable2.__neg__ = neg

In [87]:
class Sub(Function2):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

In [88]:
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

In [89]:
Variable2.__sub__ = sub
Variable2.__rsub__ = rsub

In [90]:
class Div(Function2):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

In [91]:
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

In [92]:
Variable2.__truediv__ = div
Variable2.__rtruediv__ = rdiv

In [93]:
class Pow(Function2):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

In [94]:
def pow(x, c):
    return Pow(c)(x)

In [95]:
Variable2.__pow__ = pow

In [96]:
x = Variable2(np.array(2.0))
y = x ** 0
print(y)

variable(1.0) from __repr__
